Jupyter notebook for updating an Excel spreadsheet used to contain the data for the Flourish data visualisation of the model output

In [1]:
# Import required modules
import pandas as pd
import datetime
import ipywidgets as widgets
import numpy as np

#Modules required for database access
import pyodbc
import sqlalchemy
from sqlalchemy import create_engine
import urllib

In [2]:
#Connect to database 'UK_General_Election' using SQlAlchemy
connection_str = "DRIVER={SQL SERVER};SERVER=DANZPOOTA;DATABASE=UK_General_Election;TRUSTED_CONNECTION=YES"
params = urllib.parse.quote_plus(connection_str)
engine = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)
conn = engine.connect()

In [3]:
# Parties lists
AllPartiesList = [i[0] for i in engine.execute("select PartyAbbreviation from Parties")]
GBPartiesList = ['Lab','Con','LD','Reform','Green','SNP','PC','Other']
GBPartiesAtoZList = ['Con','Green','Lab','LD','Other','PC','Reform','SNP']
UKPartiesList = ['Lab','Con','LD','Reform','SNP','PC','DUP','SF','SDLP','APNI','UUP','TUV','Green','Other']

In [4]:
# Regions lists
ITL1RegionList = [i[0] for i in engine.execute("SELECT RegionName FROM RegionRegionTypes WHERE RegionType = 'ITL1Region'")]
GBRegionsList = ['East England', 'East Midlands', 'London', 'North East England', 'North West England', 'Scotland', 'South East England', 'South West England', 'Wales', 'West Midlands', 'Yorkshire and The Humber']

In [5]:
ConLabBattlegroundYList = [1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 7, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1, 2, 3, 4, 5, 6, 7, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1, 2, 3, 1, 2, 1, 2, 3, 4, 5, 1, 2, 1, 1]

In [6]:
ConLDBattlegroundYList = [1, 2, 1, 2, 1, 2, 3, 4, 1, 1, 2, 3, 1, 1, 2, 3, 1, 2, 3, 4, 1, 2, 3, 1, 1, 2, 3, 1, 2, 3, 4, 5, 6, 7, 1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 1, 2, 3, 4, 1, 2, 3, 4, 5, 1, 2, 1, 2, 3, 4, 1, 2, 3, 4, 5, 6, 7, 1, 2, 3, 4, 5, 1, 2, 3, 1, 2]

In [7]:
SNPBattlegroundYList = [1, 1, 1, 2, 3, 4, 5, 1, 1, 2, 1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 1, 2, 1, 2, 3, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 6, 1, 2, 1, 1]

In [8]:
# Determine when the latest CORE prediction was (date) and its ID in the database
LatestPredictionDataQuery = "SELECT TOP(1) ElectionPredictionDate, ElectionPredictionID FROM ElectionPredictionMeta WHERE ElectionPredictionID LIKE '%Core%' ORDER BY ElectionPredictionDate DESC"
LatestPrediction_df = pd.read_sql(LatestPredictionDataQuery,conn)
PredictionDate = LatestPrediction_df.loc[0,'ElectionPredictionDate']
PredictionID = LatestPrediction_df.loc[0,'ElectionPredictionID']

In [9]:
PredictionID

'2024070420240703CoreComboPollElectionAlogrithm_1_0_20240703'

In [10]:
# Determine when the latest MRP prediction was (date) and its ID in the database
LatestMRPPredictionDataQuery = "SELECT TOP(1) ElectionPredictionDate, ElectionPredictionID FROM ElectionPredictionMeta WHERE ElectionPredictionID LIKE '%MRP%' ORDER BY ElectionPredictionDate DESC"
LatestMRPPrediction_df = pd.read_sql(LatestMRPPredictionDataQuery,conn)
MRPPredictionDate = LatestMRPPrediction_df.loc[0,'ElectionPredictionDate']
MRPPredictionID = LatestMRPPrediction_df.loc[0,'ElectionPredictionID']

In [11]:
MRPPredictionID

'2024070420240703MRPComboPollElectionAlogrithm_1_0_20240703'

In [12]:
PredictionCandidatesQuery = """SELECT con.ONSID, can.Constituency, Can.Party, epc.VoteShare*100 AS 'Predicted Share', epc.VoteShare*100-can.PreviousShare*100 As 'Change', can.PreviousShare*100 AS 'Previous Share' FROM ElectionPredictionCandidates AS epc
INNER JOIN Candidates AS can ON can.CandidateID = epc.CandidateID
INNER JOIN Constituencies AS con ON con.ConstituencyName = can.Constituency
WHERE ElectionPredictionID = '<PredictionID>'
ORDER BY can.Constituency ASC, epc.VoteShare DESC"""

In [13]:
PredictionConstituenciesQuery = """SELECT con.ONSID, epc.Constituency, epc.GAIN AS 'Gains', epc.LOSS AS 'Losses', epc.WinningParty AS 'Winning Party', epc.SecondParty AS 'Second Party',
epc.VoteShare*100 AS 'Winning Vote Share', epc.Majority*100 As 'Majority', epc.PreviousWinner As '2019 Winner (Notional)', epc.Swing*100 As 'Swing'
FROM ElectionPredictionConstituencies AS epc
INNER JOIN Constituencies AS con ON con.ConstituencyName = epc.Constituency
WHERE ElectionPredictionID = '<PredictionID>'"""

In [14]:
PredictionCandidatesQuery = PredictionCandidatesQuery.replace("<PredictionID>",PredictionID)
PredictionConstituenciesQuery = PredictionConstituenciesQuery.replace("<PredictionID>",PredictionID)

In [15]:
Candidates_df = pd.read_sql(PredictionCandidatesQuery,conn)
Constituencies_df = pd.read_sql(PredictionConstituenciesQuery,conn)

In [16]:
MRPPredictionConstituenciesQuery = """SELECT epc.Constituency, epc.GAIN AS 'MRP Gains', epc.WinningParty AS 'MRP Winning Party' FROM ElectionPredictionConstituencies AS epc
INNER JOIN Constituencies AS con ON con.ConstituencyName = epc.Constituency
WHERE ElectionPredictionID = '<MRPPredictionID>'"""

In [17]:
MRPPredictionConstituenciesQuery = MRPPredictionConstituenciesQuery.replace("<MRPPredictionID>",MRPPredictionID)
MRPConstituencies_df = pd.read_sql(MRPPredictionConstituenciesQuery,conn)
Constituencies_df = Constituencies_df.merge(MRPConstituencies_df, how='left', on='Constituency')

In [18]:
PredictionSeatsTimeseriesQuery = """SELECT epm.ElectionPredictionDate, epo.Constituencies AS '<Party>' FROM ElectionPredictionOverall AS epo
INNER JOIN ElectionPredictionMeta AS epm ON epm.ElectionPredictionID = epo.ElectionPredictionID
WHERE epo.Party = '<Party>' AND epm.ElectionAnalysisAlgorithm LIKE '%Core%'
ORDER BY epm.ElectionPredictionDate DESC"""

In [19]:
MRPPredictionSeatsTimeseriesQuery = """SELECT epm.ElectionPredictionDate, epo.Constituencies AS 'MRP <Party>' FROM ElectionPredictionOverall AS epo
INNER JOIN ElectionPredictionMeta AS epm ON epm.ElectionPredictionID = epo.ElectionPredictionID
WHERE epo.Party = '<Party>' AND epm.ElectionAnalysisAlgorithm LIKE '%MRP%'
ORDER BY epm.ElectionPredictionDate DESC"""

In [20]:
PredictionShareTimeseriesQuery = """SELECT epm.ElectionPredictionDate, epo.VoteShare*100 AS '<Party>' FROM ElectionPredictionOverall AS epo
INNER JOIN ElectionPredictionMeta AS epm ON epm.ElectionPredictionID = epo.ElectionPredictionID
WHERE epo.Party = '<Party>' AND epm.ElectionAnalysisAlgorithm LIKE '%Core%'
ORDER BY epm.ElectionPredictionDate DESC"""

In [21]:
MRPPredictionShareTimeseriesQuery = """SELECT epm.ElectionPredictionDate, epo.VoteShare*100 AS 'MRP <Party>' FROM ElectionPredictionOverall AS epo
INNER JOIN ElectionPredictionMeta AS epm ON epm.ElectionPredictionID = epo.ElectionPredictionID
WHERE epo.Party = '<Party>' AND epm.ElectionAnalysisAlgorithm LIKE '%MRP%'
ORDER BY epm.ElectionPredictionDate DESC"""

In [22]:
# Create the SeatsTimeseries_df and ShareTimeseries_df

# Create initial Timeseries_df just from the list of dates
SeatsTimeseries_df = pd.read_sql("SELECT ElectionPredictionDate FROM ElectionPredictionMeta WHERE ElectionAnalysisAlgorithm LIKE '%Core%' ORDER BY ElectionPredictionDate DESC",conn)
ShareTimeseries_df = SeatsTimeseries_df.copy()

for CurrentParty in GBPartiesList:
    PredictionSeatsTimeseriesQueryParty = PredictionSeatsTimeseriesQuery.replace('<Party>',CurrentParty)
    MergeSeatsTimeSeries_df = pd.read_sql(PredictionSeatsTimeseriesQueryParty,conn)
    SeatsTimeseries_df = SeatsTimeseries_df.merge(MergeSeatsTimeSeries_df, how='left', on='ElectionPredictionDate')
    
    PredictionShareTimeseriesQueryParty = PredictionShareTimeseriesQuery.replace('<Party>',CurrentParty)
    MergeShareTimeSeries_df = pd.read_sql(PredictionShareTimeseriesQueryParty,conn)
    ShareTimeseries_df = ShareTimeseries_df.merge(MergeShareTimeSeries_df, how='left', on='ElectionPredictionDate')
    
ShareTimeseries_df['Con to Lab Swing'] = ((ShareTimeseries_df['Lab'] - 32.0766315637172) - (ShareTimeseries_df['Con'] - 43.6089495387957))/2

SeatsTimeseries_df['Lab Majority'] = (SeatsTimeseries_df['Lab'] - 325) * 2

In [23]:
# Create the MRP SeatsTimeseries_df and ShareTimeseries_df

# Create initial Timeseries_df just from the list of dates
MRPSeatsTimeseries_df = pd.read_sql("SELECT ElectionPredictionDate FROM ElectionPredictionMeta WHERE ElectionAnalysisAlgorithm LIKE '%MRP%' ORDER BY ElectionPredictionDate DESC",conn)
MRPShareTimeseries_df = MRPSeatsTimeseries_df.copy()

for CurrentParty in GBPartiesList:
    MRPPredictionSeatsTimeseriesQueryParty = MRPPredictionSeatsTimeseriesQuery.replace('<Party>',CurrentParty)
    MRPMergeSeatsTimeSeries_df = pd.read_sql(MRPPredictionSeatsTimeseriesQueryParty,conn)
    MRPSeatsTimeseries_df = MRPSeatsTimeseries_df.merge(MRPMergeSeatsTimeSeries_df, how='left', on='ElectionPredictionDate')
    
    MRPPredictionShareTimeseriesQueryParty = MRPPredictionShareTimeseriesQuery.replace('<Party>',CurrentParty)
    MRPMergeShareTimeSeries_df = pd.read_sql(MRPPredictionShareTimeseriesQueryParty,conn)
    MRPShareTimeseries_df = MRPShareTimeseries_df.merge(MRPMergeShareTimeSeries_df, how='left', on='ElectionPredictionDate')
    
MRPShareTimeseries_df['MRP Con to Lab Swing'] = ((MRPShareTimeseries_df['MRP Lab'] - 32.0766315637172) - (MRPShareTimeseries_df['MRP Con'] - 43.6089495387957))/2

MRPSeatsTimeseries_df['MRP Lab Majority'] = (MRPSeatsTimeseries_df['MRP Lab'] - 325) * 2

In [24]:
ShareTimeseries_df = ShareTimeseries_df.merge(MRPShareTimeseries_df, how='left', on='ElectionPredictionDate')
SeatsTimeseries_df = SeatsTimeseries_df.merge(MRPSeatsTimeseries_df, how='left', on='ElectionPredictionDate')

In [25]:
UKShareQuery = """SELECT can.Party,
CAST(SUM(epcan.VoteShare)*100 AS FLOAT) / SUM(SUM(epcan.VoteShare)) OVER() AS 'VoteShare'
FROM ElectionPredictionCandidates AS epcan
INNER JOIN Candidates AS can ON can.CandidateID = epcan.CandidateID
INNER JOIN Constituencies AS con ON con.ConstituencyName = can.Constituency
WHERE ElectionPredictionID = '<ElectionPredictionID>'
GROUP BY can.Party
ORDER BY can.Party"""

UKShareQuery = UKShareQuery.replace('<ElectionPredictionID>',PredictionID)

In [26]:
UKSeatsQuery = """SELECT par.PartyAbbreviation,
Count(epcon.WinningParty) AS 'Seats'
FROM Parties AS par
LEFT JOIN ElectionPredictionConstituencies AS epcon ON par.PartyAbbreviation = epcon.WinningParty
AND epcon.ElectionPredictionID = '<ElectionPredictionID>'
GROUP BY par.PartyAbbreviation
ORDER BY par.PartyAbbreviation"""

UKSeatsQuery = UKSeatsQuery.replace('<ElectionPredictionID>',PredictionID)

In [27]:
GBShareQuery = """SELECT can.Party,
CAST(SUM(epcan.VoteShare)*100 AS FLOAT) / SUM(SUM(epcan.VoteShare)) OVER() AS 'VoteShare'
FROM ElectionPredictionCandidates AS epcan
INNER JOIN Candidates AS can ON can.CandidateID = epcan.CandidateID
INNER JOIN Constituencies AS con ON con.ConstituencyName = can.Constituency
WHERE ElectionPredictionID  = '<ElectionPredictionID>'
AND con.ITL1Region != 'Northern Ireland'
GROUP BY can.Party
ORDER BY can.Party"""

GBShareQuery = GBShareQuery.replace('<ElectionPredictionID>',PredictionID)

In [28]:
GBSeatsQuery = """SELECT par.PartyAbbreviation AS Party,
Count(epcon.WinningParty) AS 'Seats'
FROM ElectionPredictionConstituencies AS epcon
INNER JOIN Parties AS par ON par.PartyAbbreviation = epcon.WinningParty
INNER JOIN Constituencies AS con ON con.ConstituencyName = epcon.Constituency
WHERE con.ITL1Region != 'Northern Ireland' AND epcon.ElectionPredictionID = '<ElectionPredictionID>'
GROUP BY par.PartyAbbreviation
ORDER BY par.PartyAbbreviation"""

GBSeatsQuery = GBSeatsQuery.replace('<ElectionPredictionID>',PredictionID)

In [29]:
RegionalShareQuery = """SELECT can.Party,
CAST(SUM(epcan.VoteShare)*100 AS FLOAT) / SUM(SUM(epcan.VoteShare)) OVER() AS 'VoteShare'
FROM ElectionPredictionCandidates AS epcan
INNER JOIN Candidates AS can ON can.CandidateID = epcan.CandidateID
INNER JOIN Constituencies AS con ON con.ConstituencyName = can.Constituency
WHERE con.ITL1Region = '<Region>' AND ElectionPredictionID = '<ElectionPredictionID>'
GROUP BY con.ITL1Region, can.Party
ORDER BY can.Party"""

RegionalShareQuery = RegionalShareQuery.replace('<ElectionPredictionID>',PredictionID)

In [30]:
RegionalSeatsQuery = """SELECT par.PartyAbbreviation AS Party,
Count(epcon.WinningParty) AS 'Seats'
FROM ElectionPredictionConstituencies AS epcon
INNER JOIN Parties AS par ON par.PartyAbbreviation = epcon.WinningParty
INNER JOIN Constituencies AS con ON con.ConstituencyName = epcon.Constituency
WHERE con.ITL1Region = '<Region>' AND epcon.ElectionPredictionID = '<ElectionPredictionID>'
GROUP BY par.PartyAbbreviation
ORDER BY par.PartyAbbreviation"""

RegionalSeatsQuery = RegionalSeatsQuery.replace('<ElectionPredictionID>',PredictionID)

In [31]:
UKSharePreT_df = pd.read_sql(UKShareQuery,conn)
UKShareList = UKSharePreT_df['VoteShare'].tolist()

UKSharePre_df = pd.DataFrame([UKShareList], columns=AllPartiesList)
UKShare_df = pd.DataFrame(columns=UKPartiesList)

UKShare_df = pd.concat([UKShare_df,UKSharePre_df],axis=0)
UKShare_df['Region'] = 'UK'

In [32]:
UKSeatsPreT_df = pd.read_sql(UKSeatsQuery,conn)
UKSeatsList = UKSeatsPreT_df['Seats'].tolist()

UKSeatsPre_df = pd.DataFrame([UKSeatsList], columns=AllPartiesList)
UKSeats_df = pd.DataFrame(columns=UKPartiesList)

UKSeats_df = pd.concat([UKSeats_df,UKSeatsPre_df],axis=0)
UKSeats_df['Region'] = 'UK'

In [33]:
# Create GB share data
GBSharePreT_df = pd.read_sql(GBShareQuery,conn)
GBShareList = GBSharePreT_df['VoteShare'].tolist()
GBQueryPartiesList = GBSharePreT_df['Party'].tolist()

GBSharePre_df = pd.DataFrame([GBShareList], columns=GBQueryPartiesList)
GBShare_df = pd.DataFrame(columns=GBPartiesList)

GBShare_df = pd.concat([GBShare_df,GBSharePre_df],axis=0)
GBShare_df['Region'] = 'GB'

In [34]:
# Create GB seats data
GBSeatsPreT_df = pd.read_sql(GBSeatsQuery,conn)
GBSeatsList = GBSeatsPreT_df['Seats'].tolist()
GBQueryPartiesList = GBSeatsPreT_df['Party'].tolist()

GBSeatsPre_df = pd.DataFrame([GBSeatsList], columns=GBQueryPartiesList)
GBSeats_df = pd.DataFrame(columns=GBPartiesList)

GBSeats_df = pd.concat([GBSeats_df,GBSeatsPre_df],axis=0)
GBSeats_df['Region'] = 'GB'

In [35]:
# Now cycle through every ITL1 region to get the vote shares

#Create new df
UKShare_df = pd.concat([UKShare_df,GBShare_df],axis=0)
UKSeats_df = pd.concat([UKSeats_df,GBSeats_df],axis=0)

AllRegionShare_df = UKShare_df.copy()
AllRegionSeats_df = UKSeats_df.copy()

for CurrentRegion in ITL1RegionList:
    RegionalShareQueryReplaced = RegionalShareQuery.replace('<Region>',CurrentRegion)
    RegionalSharePreT_df = pd.read_sql(RegionalShareQueryReplaced,conn)
    RegionalSharePartiesList = RegionalSharePreT_df['Party'].tolist()
    RegionalShareSharesList = RegionalSharePreT_df['VoteShare'].tolist()
    RegionalShare_df = pd.DataFrame([RegionalShareSharesList],columns=RegionalSharePartiesList)
    RegionalShare_df['Region'] = CurrentRegion
    AllRegionShare_df = pd.concat([AllRegionShare_df,RegionalShare_df],axis=0)
    
    RegionalSeatsQueryReplaced = RegionalSeatsQuery.replace('<Region>',CurrentRegion)
    RegionalSeatsPreT_df = pd.read_sql(RegionalSeatsQueryReplaced,conn)
    RegionalSeatsPartiesList = RegionalSeatsPreT_df['Party'].tolist()
    RegionalSeatsSeatssList = RegionalSeatsPreT_df['Seats'].tolist()
    RegionalSeats_df = pd.DataFrame([RegionalSeatsSeatssList],columns=RegionalSeatsPartiesList)
    RegionalSeats_df['Region'] = CurrentRegion
    AllRegionSeats_df = pd.concat([AllRegionSeats_df,RegionalSeats_df],axis=0)

AllRegionShare_df.reset_index(drop=True,inplace=True)
AllRegionShare_df = AllRegionShare_df.fillna(0)

AllRegionSeats_df.reset_index(drop=True,inplace=True)
AllRegionSeats_df = AllRegionSeats_df.fillna(0)

In [36]:
# Determine the swings between different parties
filename = "C:\\Users\\danmu\\Documents\\Elections\\2024_Python\\Flourish_Historical.xlsx"
HistoricalRegionShare_df = pd.read_excel(filename, sheet_name='Share')

AllSwings_df = AllRegionShare_df.merge(HistoricalRegionShare_df, how='left', on='Region')
AllSwings_df['Con To Lab Swing'] = ((AllSwings_df['Lab_x']-AllSwings_df['Lab_y'])-(AllSwings_df['Con_x']-AllSwings_df['Con_y']))/2
AllSwings_df['Con To Reform Swing'] = ((AllSwings_df['Reform_x']-AllSwings_df['Reform_y'])-(AllSwings_df['Con_x']-AllSwings_df['Con_y']))/2
AllSwings_df['Con To LD Swing'] = ((AllSwings_df['LD_x']-AllSwings_df['LD_y'])-(AllSwings_df['Con_x']-AllSwings_df['Con_y']))/2
AllSwings_df['SNP To Lab Swing'] = ((AllSwings_df['Lab_x']-AllSwings_df['Lab_y'])-(AllSwings_df['SNP_x']-AllSwings_df['SNP_y']))/2

ConToLabSwingPreT_df = AllSwings_df[['Region','Con To Lab Swing']].copy()
ConToLabSwingPreT_df.set_index('Region',inplace=True)
ConToLabSwing_df = ConToLabSwingPreT_df.transpose()

ConToLDSwingPreT_df = AllSwings_df[['Region','Con To LD Swing']].copy()
ConToLDSwingPreT_df.set_index('Region',inplace=True)
ConToLDSwing_df = ConToLDSwingPreT_df.transpose()

ConToReformSwingPreT_df = AllSwings_df[['Region','Con To Reform Swing']].copy()
ConToReformSwingPreT_df.set_index('Region',inplace=True)
ConToReformSwing_df = ConToReformSwingPreT_df.transpose()

In [37]:
SNPToLabSwingPreT_df = AllSwings_df[['Region','SNP To Lab Swing']].copy()
SNPToLabSwingPreT_df.set_index('Region',inplace=True)
SNPToLabSwing_df = SNPToLabSwingPreT_df.transpose()
SNPToLabSwing_df = SNPToLabSwing_df['Scotland'].copy()
SNPToLabSwing_df 

SNP To Lab Swing    14.223738
Name: Scotland, dtype: float64

In [38]:
ConLabBatQuery = """SELECT con.ConstituencyName AS 'Constituency', con.ITL1Region, con.MajorityShare, epc.WinningParty AS 'Winning Party' FROM Constituencies AS con
INNER JOIN ElectionPredictionConstituencies AS epc ON epc.Constituency = con.ConstituencyName
WHERE epc.ElectionPredictionID = '<ElectionPredictionID>'
AND con.FirstParty = 'Con' AND con.SecondParty = 'Lab'
ORDER BY con.MajorityShare ASC"""

ConLabBatQuery = ConLabBatQuery.replace('<ElectionPredictionID>',PredictionID)

In [39]:
ConLDBatQuery = """SELECT con.ConstituencyName AS 'Constituency', con.ITL1Region, con.MajorityShare, epc.WinningParty AS 'Winning Party' FROM Constituencies AS con
INNER JOIN ElectionPredictionConstituencies AS epc ON epc.Constituency = con.ConstituencyName
WHERE epc.ElectionPredictionID = '<ElectionPredictionID>'
AND con.FirstParty = 'Con' AND con.SecondParty = 'LD'
ORDER BY con.MajorityShare ASC"""

ConLDBatQuery = ConLDBatQuery.replace('<ElectionPredictionID>',PredictionID)

In [40]:
SNPBatQuery = """SELECT con.ConstituencyName AS 'Constituency', rc.RegionName AS 'Scotland Region', con.MajorityShare, epc.WinningParty AS 'Winning Party' FROM Constituencies AS con
INNER JOIN ElectionPredictionConstituencies AS epc ON epc.Constituency = con.ConstituencyName
INNER JOIN RegionConstituencies AS RC ON rc.ConstituencyName = con.ConstituencyName
INNER JOIN Regions ON Regions.RegionName = rc.RegionName
INNER JOIN RegionRegionTypes AS rrt ON rrt.RegionName = Regions.RegionName
WHERE rrt.RegionType = 'ScotlandRegion'
AND epc.ElectionPredictionID = '<ElectionPredictionID>'
AND con.FirstParty = 'SNP'
ORDER BY con.MajorityShare ASC"""

SNPBatQuery = SNPBatQuery.replace('<ElectionPredictionID>',PredictionID)

In [41]:
ConLabBat_df = pd.read_sql(ConLabBatQuery,conn)
ConLabBat_df['RequiredSwing'] = ConLabBat_df['MajorityShare']*50
ConLabBat_df['X-Value'] = ConLabBat_df['RequiredSwing'].apply(np.floor)
ConLabBat_df['Y-Value'] = ConLabBattlegroundYList
ConLabBat_df.sort_values(by=['ITL1Region'],inplace=True)

In [42]:
ConLDBat_df = pd.read_sql(ConLDBatQuery,conn)
ConLDBat_df['RequiredSwing'] = ConLDBat_df['MajorityShare']*50
ConLDBat_df['X-Value'] = ConLDBat_df['RequiredSwing'].apply(np.floor)
ConLDBat_df['Y-Value'] = ConLDBattlegroundYList
ConLDBat_df.sort_values(by=['ITL1Region'],inplace=True)

In [43]:
SNPBat_df = pd.read_sql(SNPBatQuery,conn)
SNPBat_df['RequiredSwing'] = SNPBat_df['MajorityShare']*50
SNPBat_df['X-Value'] = SNPBat_df['RequiredSwing'].apply(np.floor)
SNPBat_df['Y-Value'] = SNPBattlegroundYList
SNPBat_df.sort_values(by=['Scotland Region'],inplace=True)

In [44]:
CoreOverallQueryRaw = """SELECT Party, Constituencies AS 'Seats', VoteShare*100 AS 'Share' FROM ElectionPredictionOverall
WHERE ElectionPredictionID = '<PredictionID>'
ORDER BY Constituencies DESC"""

CoreOverallQuery = CoreOverallQueryRaw.replace('<PredictionID>',PredictionID)

In [45]:
MRPOverallQueryRaw = """SELECT Party, Constituencies AS 'MRP Seats', VoteShare*100 AS 'MRP Share' FROM ElectionPredictionOverall
WHERE ElectionPredictionID = '<MRPPredictionID>'
ORDER BY Constituencies DESC"""

MRPOverallQuery = MRPOverallQueryRaw.replace('<MRPPredictionID>',MRPPredictionID)

In [46]:
CoreOverall_df = pd.read_sql(CoreOverallQuery,conn)
MRPOverall_df = pd.read_sql(MRPOverallQuery,conn)
OverallTable_df = CoreOverall_df.merge(MRPOverall_df, how='left', on='Party')
OverallTable_df['Seats Delta'] = OverallTable_df['Seats'] - OverallTable_df['MRP Seats']
OverallTable_df['Share Delta'] = OverallTable_df['Share'] - OverallTable_df['MRP Share']

In [47]:
AllConstTable_df = Constituencies_df[['Constituency','2019 Winner (Notional)','Winning Party','MRP Winning Party']].copy()
AllConstTable_df.rename(columns={"2019 Winner (Notional)": "Previous Winner"})
AllConstTable_df.sort_values(by=['Constituency'],inplace=True)

In [48]:
with pd.ExcelWriter(r'C:\Users\danmu\Documents\Elections\2024_Python\Flourish_Export.xlsx') as writer:  
    OverallTable_df.to_excel(writer, sheet_name='Overall',index=False)
    Candidates_df.to_excel(writer, sheet_name='Candidates',index=False)
    Constituencies_df.to_excel(writer, sheet_name='Constituencies',index=False)
    SeatsTimeseries_df.to_excel(writer, sheet_name='SeatsTimeseries',index=False)
    ShareTimeseries_df.to_excel(writer, sheet_name='ShareTimeseries',index=False)
    AllRegionShare_df.to_excel(writer, sheet_name='Share',index=False)
    AllRegionSeats_df.to_excel(writer, sheet_name='Seats',index=False)
    LatestPrediction_df.to_excel(writer, sheet_name='LatestPrediction',index=False)
    ConToLabSwing_df.to_excel(writer, sheet_name='ConToLabSwing',index=True)
    ConToLDSwing_df.to_excel(writer, sheet_name='ConToLDSwing',index=True)
    ConToReformSwing_df.to_excel(writer, sheet_name='ConToReformSwing',index=True)
    SNPToLabSwing_df.to_excel(writer, sheet_name='SNPtoLabSwing',index=True)
    ConLabBat_df.to_excel(writer, sheet_name='ConLabBat',index=True)
    ConLDBat_df.to_excel(writer, sheet_name='ConLDBat',index=True)
    SNPBat_df.to_excel(writer, sheet_name='SNPBat',index=True)
    AllConstTable_df.to_excel(writer, sheet_name='AllConstTable',index=True)

In [49]:
#Close the connection with the database
conn.close()